# Prototype

In [81]:
import numpy as np

### Gameplay

In [125]:
class Gameplay:
    def __init__(self, n_rows, n_cols):
        self.columns = n_cols
        self.rows = n_rows
        self.rules = []
        self.tiles = np.zeros((n_rows, n_cols), dtype=object)
        for i in range(n_rows):
            for j in range(n_cols):
                self.tiles[i,j] = Tile()

        self.prefixes = ['baba', 'rock', 'water', 'skull', 'wall', 'flag']
        self.suffixes = ['you', 'push', 'sink', 'defeat', 'stop', 'win']
            

    # def __repr__(self):
    #     ret_val = ""
    #     for row in range(self.rows):
    #         for col in range(self.columns):
    #             if len(self.tiles[row,col].objects) == 0:
    #                 ret_val += 'null'
    #             else:
    #                 for obj in self.tiles[row,col].objects:
    #                     if isinstance(obj, Word):
    #                         ret_val += obj.value
    #                     else:
    #                         ret_val += obj.property if obj.property != '' else 'null'
    #                     ret_val += ', '
    #             ret_val += '\\ '
    #         ret_val += '\n'
    #     return ret_val
    
    def __repr__(self):
        ret_val = ''
        for i in range(self.rows):
            for j in range(self.columns):
                tile_string = ''
                if len(self.tiles[i,j].objects) == 0:
                    tile_string += '/'
                    # tile_string += 'null'
                else:
                    for obj in self.tiles[i,j].objects:
                        if isinstance(obj, Word):
                            tile_string += obj.value.upper()
                        else:
                            tile_string += obj.property if obj.property !='' else 'null'
                        tile_string += ','
                ret_val += '{:10}'.format(tile_string)
                # ret_val += tile_string
            ret_val += '\n'
        return ret_val


    def get_rules(self):
        self.rules = []
    
        for i in range(self.rows):
            for j in range(self.columns):
                if self.tiles[i,j].find_word() == "is":
                    print("yes")
                    
                    #left -> right
                    if j-1 >= 0 and j+1 <=self.columns-1:
                        if self.tiles[i,j-1].find_word() in self.prefixes and self.tiles[i,j+1].find_word() in self.suffixes:
                            rule = Rule(self.tiles[i,j-1].find_word(), self.tiles[i,j+1].find_word())
                            self.rules += [rule]

                    #up -> down
                    if i-1 >= 0 and i+1 <=self.rows-1:
                        if self.tiles[i-1,j].find_word() in self.prefixes and self.tiles[i+1,j].find_word() in self.suffixes:
                            rule = Rule(self.tiles[i-1,j].find_word(), self.tiles[i+1,j].find_word())
                            self.rules += [rule]

    def find_block_upward(self,row,col):        
        if not self.tiles[row,col].has_you():
            return 0
        count = 1
        current_row = row - 1
        # current_row < 0 : out of map -> no need to check anymore
        if current_row < 0:
            return -1
        # current_row >= 0 check again, so the checked tile is still in the map
        while current_row >= 0 and self.tiles[current_row,col].has_push():     
            current_row -= 1
            count += 1
        #       check stop tile                        out of map
        if self.tiles[current_row,col].has_stop() or current_row < 0:
            return -1
            
        else:
            return count

            
    def move_up(self):        
        for row in range(self.rows):
            for col in range(self.columns):
                size = self.find_block_upward(row,col)
                if size > 0:
                    for current_row in range(row+1-size,row+1):
                        temp = self.tiles[current_row,col].pop_push_or_you()
                        self.tiles[current_row-1,col].objects = np.append(self.tiles[current_row-1,col].objects, [temp])
                                   
        
    def move_down(self):
        pass
    def move_left(self):
        pass
    def move_right(self):
        pass


    def push_objects(self, direction, col, row):
        pass


    def check_win(self):
        pass
    def check_lose(self):
        pass

    
    def reset_game(self):
        pass

    def load_map_level(self, file_name):
        pass

### Rule

In [83]:
class Rule:
    def __init__(self, first, second):         #first + is + second 
        self.first = first
        self.second = second
    
    def __repr__(self):
        return self.first +" is "+ self.second


### Object

In [84]:
class Object:
    def __init__(self, property=''):
        self.property = property

    def interaction(self, another_object:object):
        
        pass
        

In [85]:
class Baba(Object):        
    def interaction(self, another_object:object):
        
        pass

In [86]:
class Rock(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [87]:
class Wall(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [88]:
class Water(Object):
    def interaction(self, another_object:object):
        
        pass
     

In [89]:
class Skull(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [90]:
class Flag(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [91]:
class Word(Object):
    def __init__(self, value):        
        super().__init__('push')
        self.value = value

    def interaction(self, another_object:object):
        
        pass
        

### Tile

In [115]:
class Tile:
    def __init__(self):
        self.objects = np.array([], dtype=object)
    def add_object(self, obj : Object):
        self.objects = np.append(self.objects, [obj])

    def find_word(self):
        for obj in self.objects:
            if isinstance(obj, Word):
                return obj.value
            return ''

    def has_push(self):
        for obj in self.objects:
            if obj.property == 'push':
                return True
        return False

    def has_you(self):
        for obj in self.objects:
            if obj.property == 'you':
                return True
        return False

    def has_stop(self):
        for obj in self.objects:
            if obj.property == 'stop':
                return True
        return False

    # delete a 'push' object in the list
    # return that 'push' object to a variable
    def pop_push_or_you(self):
        position = -1
        for i in range(len(self.objects)):
            if self.objects[i].property == 'push' or self.objects[i].property == 'you' :
                position = i
                print(position)

        temp_obj = self.objects[position]
        self.objects = np.delete(self.objects, position)

        return temp_obj
    

### Unit Testing

In [93]:
test_gp = Gameplay(6,15)
test_gp.tiles[4,9].add_object(Word("is"))
# test_gp.tiles[4,9].add_object(Flag())
# test_gp.tiles[4,9].add_object(Word("is"))
test_gp.tiles[3,9].add_object(Word("baba"))
test_gp.tiles[5,9].add_object(Word("you"))
test_gp.tiles[4,9].add_object(Word("is"))
test_gp.tiles[4,8].add_object(Word("rock"))
test_gp.tiles[4,10].add_object(Word("push"))
test_gp.tiles[5,9].add_object(Word("is"))
test_gp.tiles[5,8].add_object(Word("baba"))
test_gp.tiles[5,10].add_object(Word("rock"))
test_gp.tiles[2,9].add_object(Flag())
test_gp.get_rules()


yes


In [94]:
test_gp.tiles[4,10].has_push()


True

In [95]:
test_gp.tiles[5,9].has_push()

True

In [96]:
test_gp.tiles[2,9].has_push()

False

In [97]:
print(test_gp)

/         /         /         /         /         /         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         /         /         /         /         /         /         
/         /         /         /         /         /         /         /         /         null,     /         /         /         /         /         
/         /         /         /         /         /         /         /         /         BABA,     /         /         /         /         /         
/         /         /         /         /         /         /         /         ROCK,     IS,IS,    PUSH,     /         /         /         /         
/         /         /         /         /         /         /         /         BABA,     YOU,IS,   ROCK,     /         /         /         /         



In [98]:
test_tile = Tile()
test_tile

In [99]:
print(test_gp.rules)
print(test_gp.rules[0].first)
print(test_gp.rules[0].second)
print(test_gp.rules[0])

[rock is push, baba is you]
rock
push
rock is push


### Movement testing

In [127]:
test_mm = Gameplay(7,7)
test_mm.tiles[3,3].add_object(Rock("you"))
test_mm.tiles[2,3].add_object(Word("is"))
test_mm.tiles[1,3].add_object(Word("defeat"))

test_mm.tiles[2,4].add_object(Rock("you"))
test_mm.tiles[1,4].add_object(Word("is"))
test_mm.tiles[0,4].add_object(Word("defeat"))

In [128]:
print(test_mm)

/         /         /         /         DEFEAT,   /         /         
/         /         /         DEFEAT,   IS,       /         /         
/         /         /         IS,       you,      /         /         
/         /         /         you,      /         /         /         
/         /         /         /         /         /         /         
/         /         /         /         /         /         /         
/         /         /         /         /         /         /         



In [129]:
test_mm.move_up()

3
0
0
0


In [130]:
print(test_mm)

/         /         /         DEFEAT,   DEFEAT,   /         /         
/         /         /         IS,       IS,       /         /         
/         /         /         you,      you,      /         /         
/         /         /         /         /         /         /         
/         /         /         /         /         /         /         
/         /         /         /         /         /         /         
/         /         /         /         /         /         /         

